In [ ]:
from hydromt_sfincs import SfincsModel
from os.path import join, isfile, basename, isdir, dirname
import xarray as xr
import numpy as np
import glob
import hydromt

In [ ]:
events = {'idai': ['20190319', '20190320'], 'eloise':['20210125','20210126']}
tslice_max = {'idai': ('20190313', '20190321'), 'eloise':('20210120', '20210127')}
root0 = r'../../3_models/SFINCS2'
mod0 = SfincsModel(join(root0, '00_base_100m'), mode='r')

def check_finished(root):
    finished = False
    if isfile(join(root, 'sfincs.log')):
        with open(join(root, 'sfincs.log'), 'r') as f:
            finished = np.any(['Simulation is finished' in l for l in f.readlines()])
    return finished

In [ ]:
# read and combine SFINCS outputs
postfix, i = '_100m', 0
runs = glob.glob(join(root0, '*/*/','sfincs_map.nc'))
encoding = {'flddph': {'zlib': True}}

# mod0 = SfincsModel(runs[0], mode='r')
# print(runs)

for event, dates in events.items():
    runs = glob.glob(join(root0, '0*', f'{event}*', 'sfincs_map.nc'))
    print(event)
    da_lst, da_lst1, names = [], [], []
    for run in runs:
        root = dirname(run)
        if not basename(dirname(root)).startswith(str(i)): continue
        if not check_finished(root): 
            print(f'{root} NOT FINISHED / FAILED')
            continue
        mod = SfincsModel(root, mode='r')
        key = basename(root).strip(f'{event}')
        name = f'{basename(dirname(root))}{key}'
        # print(name)
        mod.read_results()
        hmax = np.maximum(0, mod.results['zsmax'] - mod.results['zb'])
        hmax = hmax.fillna(0).where(mod.results['zb'].notnull(),-9999)
        hmax.raster.set_nodata(-9999)
        # hmax = hmax.raster.reproject_like(mod0.staticmaps, 'min')
        # save geotif
        # hmax.raster.flipud().raster.to_raster(join(mod.root, 'gis', 'hmax.tif'), compress='lzw')
        # append to list
        da_lst.append(hmax.sel(timemax=slice(*tslice_max[event])).rename({'timemax': 'time'}).rename('flddph'))
        # da_lst.append(hmax.sel(timemax=dates).rename({'timemax': 'time'}).rename('flddph'))
        # da_lst1.append(hmax.sel(timemax=slice(*tslice_max[event])).max('timemax').rename('flddph'))
        names.append(name)
    # save combined output to nc
    da = xr.concat(da_lst, dim='run')
    da['run'] = xr.IndexVariable('run', names)
    da.to_netcdf(join(root0, f'flddph_{event}{postfix}_v2.nc'), encoding=encoding)
    # save combined output to nc
    # da = xr.concat(da_lst1, dim='run')
    # da['run'] = xr.IndexVariable('run', names)
    # da.to_netcdf(join(root0, f'flddph_{event}{postfix}_max_v2.nc'), encoding=encoding)
    # break

In [ ]:
# read and combine CMF outputs
import pandas as pd
root = r'../../3_models/CMF'
encoding = {'flddph': {'zlib': True}}

for event, _ in events.items():
    da_lst, names = [], []
    dates = pd.date_range(*tslice_max[event], freq='D').strftime('%Y%m%d')
    for res in ['03min', '06min']:
        runs = [path for path in glob.glob(join(root, res, '0*')) if isdir(path)]
        for run in runs:
            da_lst0 = []
            for date in dates:
                fn = join(run, f'flddph_{date}.tif')
                if not isfile(fn): 
                    print('missing', fn)
                    continue
                hmax = hydromt.open_raster(fn).raster.reproject_like(mod0.staticmaps)
                da_lst0.append(hmax.rename('flddph'))
            da0 = xr.concat(da_lst0, dim='time')
            da0['time'] = xr.IndexVariable('time', pd.to_datetime(dates))
            da_lst.append(da0)
            names.append(f'{basename(run)}_{res}')
    # save combined output to nc
    print(names)
    da = xr.concat(da_lst, dim='run')
    da['run'] = xr.IndexVariable('run', names)
    da.to_netcdf(join(root, f'flddph_{event}_v2.nc'), encoding=encoding)